In [5]:
# IMPORT
import spacy
# import scispacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
import tqdm
import pandas as pd
import re
import networkx as nx
import matplotlib.pyplot as plt
from spacy.lang.en import English # updated
nlp = spacy.load('en_core_web_sm')
from data import *
from app import *

In [20]:
txt2

'An engineer had to plan the construction of an artificial lake to produce electric energy. To feed the lake he thought to build a unique wide canal collecting water coming from a near valley. However, a mason pointed out that during the flood periods the stream of water flowing along the canal might be too strong and might damage the surrounding areas; by contrast, during the drought periods a unique stream of water might be insufficient to feed the lake. In order to avoid these mishaps, the mason suggested to build, instead of a unique wide canal, four small canals whose total flow was the same as the unique wide canal previously planned. These small canals were placed around the lake so that they conveyed water coming from four different valleys. In this way only small amounts of water could flow in each canal and thus during flood periods dangerous overflowing might not occur. At the same time, the lake was fed by water from various belts, so that also during drought periods it was

In [30]:
# Get root of sentence
def n_chunk(sent):
    roots = ''
    doc = nlp(sent)
    for chunk in doc.noun_chunks:
        print(f"\nChunk Text: {chunk.text}\n-> Root: {chunk.root.text}\n-> Arc label:{chunk.root.dep_}\n-> Root head: {chunk.root.head.text}\n")
        roots += chunk.root.text+ ' ' +chunk.root.head.text+' '

    return roots

In [50]:
def noun_chunks_sent(sentence):
    '''
    Take in single sntence and output
    nound chunks
    '''
    chunks = []
    sent = nlp(sentence)
    for chunk in sent.noun_chunks:
        print(chunk.left())
        chunks.append(chunk)
#         print("Chunk: ",chunk)
    return chunks

In [52]:
chunks = noun_chunks_sent(txt2)
len(chunks)
chunks

AttributeError: 'spacy.tokens.span.Span' object has no attribute 'left'

In [61]:
doc = nlp(txt2)
# nlp = spacy.load("en_core_web_sm")
# doc = nlp("bright red apples on the tree")
print(type(doc))
print([token.text for token in doc[2].lefts])  # ['bright', 'red']
print([token.text for token in doc[2].rights])  # ['on']
print(doc[2].n_lefts)  # 2
print(doc[2].n_rights)  # 1
doc[3].n_rights

<class 'spacy.tokens.doc.Doc'>
['engineer']
['plan', '.']
1
2


0

In [42]:
def noun_component(text):
    doc = nlp(text)
    print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
    print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

In [43]:
noun_component(txt2)

Noun phrases: ['An engineer', 'the construction', 'an artificial lake', 'electric energy', 'the lake', 'he', 'a unique wide canal collecting water', 'a near valley', 'a mason', 'the flood periods', 'the stream', 'water', 'the canal', 'the surrounding areas', 'contrast', 'a unique stream', 'water', 'the lake', 'order', 'these mishaps', 'the mason', 'a unique wide canal', 'four small canals', 'whose total flow', 'the unique wide canal', 'These small canals', 'the lake', 'they', 'water', 'four different valleys', 'this way', 'only small amounts', 'water', 'each canal', 'flood periods dangerous overflowing', 'the same time', 'the lake', 'water', 'various belts', 'drought periods', 'it']
Verbs: ['plan', 'produce', 'feed', 'think', 'build', 'collect', 'come', 'point', 'flow', 'may', 'may', 'damage', 'surround', 'period', 'may', 'feed', 'avoid', 'suggest', 'build', 'plan', 'place', 'convey', 'come', 'could', 'flow', 'may', 'occur', 'feed']


In [44]:
def noun_component_list(text):
    doc = nlp(text)
    chunks = list(doc.noun_chunks)
    return chunks

In [45]:
noun_component_list(txt2)

[An engineer,
 the construction,
 an artificial lake,
 electric energy,
 the lake,
 he,
 a unique wide canal collecting water,
 a near valley,
 a mason,
 the flood periods,
 the stream,
 water,
 the canal,
 the surrounding areas,
 contrast,
 a unique stream,
 water,
 the lake,
 order,
 these mishaps,
 the mason,
 a unique wide canal,
 four small canals,
 whose total flow,
 the unique wide canal,
 These small canals,
 the lake,
 they,
 water,
 four different valleys,
 this way,
 only small amounts,
 water,
 each canal,
 flood periods dangerous overflowing,
 the same time,
 the lake,
 water,
 various belts,
 drought periods,
 it]